# Multi-Agent Collaboration with Amazon Bedrock

This notebook explores the **multi-agent collaboration capability** in **Amazon Bedrock**, where specialized agents work together to solve complex tasks. The multi-agent system is orchestrated by a supervisor agent, allowing efficient task delegation and collaboration.

---

## Objectives

1. **Understand the Multi-Agent Framework in Amazon Bedrock:**
   - Learn about supervisor agents and subagents.
   - Explore collaboration modes: Supervisor Mode and Supervisor with Routing Mode.

2. **Create and Configure Agents:**
   - Define and configure subagents for specific tasks.
   - Create a supervisor agent to coordinate subagents.

3. **Build and Test Multi-Agent Collaboration:**
   - Use Amazon Bedrock to orchestrate multi-agent workflows.
   - Test with real-world examples like social media campaign management.

4. **Student Challenges:**
   - Build multi-agent systems for custom scenarios.
   - Test and debug multi-agent interactions.

---

## Concepts

### 1. **Supervisor and Subagents**
- **Supervisor Agent:** Manages task delegation, breakdown, and final response synthesis.
- **Subagents:** Specialized agents focusing on distinct tasks (e.g., content generation, analytics).

### 2. **Collaboration Modes**
- **Supervisor Mode:**
  - The supervisor analyzes input and orchestrates subagents.
  - Best for complex tasks requiring full orchestration.
  
- **Supervisor with Routing Mode:**
  - Simple tasks are routed directly to relevant subagents.
  - For ambiguous or complex queries, the system switches to supervisor mode.

### 3. **Trace and Debug**
- The **trace console** provides a timeline of subagent interactions.
- Helps debug workflows and optimize inter-agent communication.

---

## Hands-On: Multi-Agent Collaboration in Amazon Bedrock

In [ ]:
from dotenv import load_dotenv
import botocore.exceptions
import os
import json
import sys
import boto3

aws_region = "us-east-1"

load_dotenv(".env")

bedrock_runtime_client = boto3.client("bedrock-runtime", region_name=aws_region)
bedrock_management_client = boto3.client('bedrock', region_name=aws_region)
bedrock_agent_client = boto3.client('bedrock-agent', region_name=aws_region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=aws_region)
cloudformation_client = boto3.client('cloudformation', region_name=aws_region)

boto3.__version__

In [ ]:
%store -r stack_name
%store -r agent_stack_name

stack_name, agent_stack_name

In [ ]:
response_stack1 = cloudformation_client.describe_stacks(StackName=stack_name)
outputs_stack1 = response_stack1['Stacks'][0]['Outputs']

response_stack2 = cloudformation_client.describe_stacks(StackName=agent_stack_name)
outputs_stack2 = response_stack2['Stacks'][0]['Outputs']

results = {}

for output in outputs_stack1:
    results[output['OutputKey']] = output['OutputValue']
for output in outputs_stack2:
    results[output['OutputKey']] = output['OutputValue']
    
for key, value in results.items():
    print(f"{key}: {value}")

In [ ]:
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0'),
        ('Titan Text G1 - Premier', 'amazon.titan-text-premier-v1:0'),
        ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
        ('Titan Text G1 - Express', 'amazon.titan-text-express-v1')
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

### Step 1: Define Subagents

We create two subagents for a social media campaign manager:
1. **Content Strategist:** Generates creative content ideas.
2. **Engagement Predictor:** Analyzes post performance and suggests optimal posting times.

In [45]:
def create_agent(agent_name, description, model_id, instructions, agent_collaboration='DISABLED'):
    response = bedrock_agent_client.create_agent(
        agentCollaboration=agent_collaboration,
        agentName=agent_name,
        description=description,
        foundationModel=model_id,
        instruction=instructions,
        agentResourceRoleArn=results["BedrockExecutionRoleArn"]
    )
    return response['agent']

def prepare_agent(agent_id):
    response = bedrock_agent_client.prepare_agent(
        agentId=agent_id
    )
    return response

In [46]:
content_agent = create_agent(
    agent_name="content-strategist",
    description="Generates creative social media content ideas.",
    model_id=agent_foundation_model_selector.value,
    instructions=(
        "You are a content strategist. Generate creative, on-brand social media ideas "
        "aligned with campaign goals. Each suggestion should include content type, copy, and hashtags."
    )
)

In [ ]:
content_agent

In [ ]:
prepare_agent(content_agent['agentId'])

In [49]:
engagement_agent = create_agent(
    agent_name="engagement-predictor",
    description="Predicts social media post performance.",
    model_id=agent_foundation_model_selector.value,
    instructions=(
        "You are a social media analytics expert. Predict post performance and suggest optimal timing "
        "based on industry benchmarks and audience behavior."
    )
)

In [ ]:
engagement_agent

In [ ]:
prepare_agent(engagement_agent['agentId'])

### Step 2: Create Supervisor Agent

The **Supervisor Agent** orchestrates the workflow by coordinating subagents.

In [53]:
supervisor_agent = create_agent(
    agent_name="social-media-campaign-manager",
    description="Coordinates a social media campaign by combining content and analytics insights.",
    model_id=agent_foundation_model_selector.value,
    instructions="You are a campaign manager orchestrating tasks across agents to create strategic plans.",
    agent_collaboration='SUPERVISOR'
)

In [ ]:
supervisor_agent

### Step 3: Create Agent Alias

The agent alias is a prerequisite for the association operation that will occur in the next step.

In [56]:
def create_agent_alias(alias_name, agent_id, description):
    response = bedrock_agent_client.create_agent_alias(
        agentAliasName=alias_name,
        agentId=agent_id,
        description=description
    )
    return response

In [58]:
content_alias_response = create_agent_alias(
    alias_name="content-strategist-content",
    agent_id=content_agent['agentId'],
    description="Alias for the content strategist content agent."
)

In [ ]:
content_alias_response

In [59]:
engagement_alias_response = create_agent_alias(
    alias_name="content-strategist-engagement",
    agent_id=engagement_agent['agentId'],
    description="Alias for the content strategist engagement agent."
)

In [ ]:
engagement_alias_response

### Step 4: Associate Subagents with Supervisor

Use `AssociateAgentCollaborator` to link subagents with the supervisor agent.

In [64]:
def associate_collaborator(supervisor_id, collaborator_alias_arn, agent_version, collaboration_instruction, collaborator_name, relay_history):
    response = bedrock_agent_client.associate_agent_collaborator(
        agentDescriptor={
            'aliasArn': collaborator_alias_arn
        },
        agentId=supervisor_id,
        agentVersion=agent_version,
        collaborationInstruction=collaboration_instruction,
        collaboratorName=collaborator_name,
        relayConversationHistory=relay_history
    )
    return response

In [65]:
associate_collaborator_response = associate_collaborator(
    supervisor_id=supervisor_agent['agentId'],
    collaborator_alias_arn=content_alias_response['agentAlias']['agentAliasArn'],
    agent_version="DRAFT",
    collaboration_instruction="Handles content generation for social media campaigns.",
    collaborator_name="content-strategist",
    relay_history="TO_COLLABORATOR"
)

In [ ]:
associate_collaborator_response

In [67]:
associate_engagement_response = associate_collaborator(
    supervisor_id=supervisor_agent['agentId'],
    collaborator_alias_arn=engagement_alias_response['agentAlias']['agentAliasArn'],
    agent_version="DRAFT",
    collaboration_instruction="Predicts social media post performance and optimal timing.",
    collaborator_name="engagement-predictor",
    relay_history="TO_COLLABORATOR"
)

In [ ]:
associate_engagement_response

### Step 4: Test the Multi-Agent System

Invoke the **supervisor agent** with a query that requires both subagents to collaborate.

In [ ]:
prepare_agent(supervisor_agent['agentId'])

In [73]:
supervisor_alias_response = create_agent_alias(
    alias_name="supervisor-agent-content-alias",
    agent_id=supervisor_agent['agentId'],
    description="Alias for the supervisor agent managing content strategies."
)

In [ ]:
supervisor_alias_response

In [88]:
import uuid
import boto3
import pprint

pp = pprint.PrettyPrinter(indent=2)

def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    if not session_state:
        session_state = {}

    payload = {
        "inputText": query,
        "agentId": agent_id,
        "agentAliasId": alias_id,
        "sessionId": session_id,
        "enableTrace": enable_trace,
        "endSession": False,
        "sessionState": session_state
    }

    try:
        agent_response = bedrock_agent_runtime_client.invoke_agent(**payload)
        event_stream = agent_response['completion']
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                response_text = data.decode('utf8')
                return response_text
            elif 'trace' in event and enable_trace:
                pp.pprint(event['trace'])
            else:
                raise Exception(f"Unexpected event received: {event}")

    except Exception as e:
        print(f"Error while invoking agent: {e}")
        raise

In [ ]:
session_id = "multi-agent-session"

try:
    supervisor_response = invoke_agent_helper(
        query=(
            "Create a 2-week campaign for EcoTech's solar panels. Target: B2B, Key points: 30% more efficient, "
            "AI-optimized, 2-year ROI. Need: 4 posts/week on LinkedIn and Twitter."
        ),
        session_id=session_id,
        agent_id=supervisor_alias_response['agentAlias']['agentId'],
        alias_id=supervisor_alias_response['agentAlias']['agentAliasId'],
        enable_trace=False
    )

    print("Agent Response:")
    print(supervisor_response)

except Exception as e:
    print(f"Error in invoking agent helper: {e}")

## **Preparation: Setting Up the Knowledge Base (KB)**

In this section, we will create a **Knowledge Base (KB)** that will serve as the foundation for the multi-agent collaboration. The KB will store structured data and relevant insights that agents can use to generate responses and coordinate tasks effectively. This KB will be integrated into Amazon Bedrock agents for retrieval and reasoning tasks.

In [2]:
## Use the Knowledge Base (KB) Created Earlier for the Restaurant to Implement the Dining Agent.

## Challenge for Students

### Optimize Multi-Agent Workflow for a Travel Planner

**Objective:** Use a multi-agent system to collaboratively design an optimal travel itinerary tailored to user preferences.

#### **Tasks:**

1. **Attraction Agent**: Suggest attractions based on user-provided preferences such as destination, interests, and trip duration.
2. **Logistics Agent**: Plan travel routes and accommodations, ensuring the itinerary is feasible and cost-effective.
3. **Dining Agent**: Recommend dining options near attractions, tailored to user preferences (e.g., cuisine type, dietary restrictions), you can use the previously created restaurant knowledge base.
4. **Supervisor Agent**: Integrate outputs from all agents into a cohesive travel plan, ensuring compatibility and balance.

#### **Deliverable:**

- Python code to implement and integrate the multi-agent system.
- A test scenario with input (e.g., “Plan a 5-day trip to Paris for a family with kids who enjoy history and outdoor activities”).
- JSON output presenting the finalized travel itinerary, including attractions, routes, accommodations, and dining recommendations.


### **Key Takeaways:**
- **Supervised Orchestration:** Supervisor agents coordinate tasks across specialized subagents.
- **Collaboration Modes:** Choose between Supervisor and Supervisor with Routing modes.
- **Real-World Applications:** Multi-agent collaboration is powerful for complex tasks like social media management, recommendations, and travel planning.

By completing these challenges, students will gain hands-on experience in implementing and testing multi-agent workflows in Amazon Bedrock!

### Conclusion and Cleanup

To avoid incurring additional costs, it is critical to delete the infrastructure and resources provisioned for this lab after completing all exercises. This includes components like the S3 bucket, OpenSearch collection, and other CloudFormation stack resources.

You can delete the CloudFormation stack using the following code:

```python
cloudformation_client.delete_stack(StackName=stack_name)
cloudformation_client.delete_stack(StackName=agent_stack_name)
print("Stack deletion initiated.")
```

Ensure you only initiate this cleanup process after verifying that no further labs or activities require the resources.